# Finance Domain - Simplified Version

## Overview
This notebook creates Finance domain tables that integrate with shared master data from the Sales domain.

## Key Achievements
- **Domain-driven design**: Finance-specific business logic
- **Shared master data**: References Customer, Product tables from Sales domain
- **Clean integration**: Links to Sales Order data via foreign keys
- **Enterprise-ready**: Proper accounting and financial tracking

## Schema Structure
- **Finance Domain**: 4 tables (Account, Invoice, Payment, Transaction)
- **Integration**: Links to shared Customer, Product, Order tables

## Dependencies
- **Prerequisite**: Run Customer_Product_Order_Simplified_12.ipynb first
- **Shared Tables**: Customer, Product, Order (created by Sales domain)

---

In [ ]:
# Finance Domain Tables - References Shared Master Data

################################################################################################
# Verify Schema Exists
################################################################################################

# Configuration
SCHEMA_NAME = "retail_simple"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {SCHEMA_NAME}")
print(f"✅ {SCHEMA_NAME} schema ready!")

print(f"\n💰 CREATING FINANCE DOMAIN TABLES")
print("="*60)
print(f"📋 These tables reference shared master data:")
print(f"   • Customer (created by Sales domain)")
print(f"   • Product (created by Sales domain)") 
print(f"   • Order (created by Sales domain)")
print("="*60)

################################################################################################
# FINANCE DOMAIN TABLES
################################################################################################

# 1. Create Account table (Chart of Accounts)
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Account (
    AccountId STRING,
    AccountNumber STRING,
    AccountName STRING,
    AccountType STRING,
    AccountCategory STRING,
    IsActive BOOLEAN,
    CreatedDate DATE,
    CreatedBy STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Account table created!")

# 2. Create Invoice table (Bills sent to customers - Links to Sales Order)
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Invoice (
    InvoiceId STRING,
    InvoiceNumber STRING,
    CustomerId STRING,        -- FK to shared Customer table
    OrderId STRING,           -- FK to shared Order table  
    InvoiceDate DATE,
    DueDate DATE,
    SubTotal DECIMAL(18,2),
    TaxAmount DECIMAL(18,2),
    TotalAmount DECIMAL(18,2),
    InvoiceStatus STRING,
    BillingAddress STRING,
    InvoiceNotes STRING,
    IsoCurrencyCode STRING,
    CreatedTimestamp TIMESTAMP,
    CreatedBy STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Invoice table created!")

# 3. Create Payment table (Money received from customers)
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Payment (
    PaymentId STRING,
    InvoiceId STRING,         -- FK to Invoice table
    CustomerId STRING,        -- FK to shared Customer table
    PaymentDate DATE,
    PaymentAmount DECIMAL(18,2),
    PaymentMethod STRING,
    PaymentStatus STRING,
    TransactionId STRING,
    PaymentReference STRING,
    IsoCurrencyCode STRING,
    CreatedTimestamp TIMESTAMP,
    CreatedBy STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Payment table created!")

# 4. Create Transaction table (All financial movements)
create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {SCHEMA_NAME}.Transaction (
    TransactionId STRING,
    TransactionDate DATE,
    TransactionType STRING,
    Amount DECIMAL(18,2),
    Description STRING,
    FromAccountId STRING,     -- FK to Account table
    ToAccountId STRING,       -- FK to Account table
    OrderId STRING,           -- FK to shared Order table
    CustomerId STRING,        -- FK to shared Customer table
    InvoiceId STRING,         -- FK to Invoice table
    IsoCurrencyCode STRING,
    CreatedTimestamp TIMESTAMP,
    CreatedBy STRING
)
USING DELTA
"""
spark.sql(create_table_sql)
print(f"✅ {SCHEMA_NAME}.Transaction table created!")

print(f"\n✅ All Finance Domain tables created successfully!")
print(f"📊 Finance Domain tables: 4")
print(f"\n🔗 CROSS-DOMAIN INTEGRATION:")
print(f"   • Invoice.CustomerId → Customer.CustomerId (Sales domain)")
print(f"   • Invoice.OrderId → Order.OrderId (Sales domain)")
print(f"   • Transaction.CustomerId → Customer.CustomerId (Sales domain)")
print(f"   • Transaction.OrderId → Order.OrderId (Sales domain)")
print(f"   • Payment.InvoiceId → Invoice.InvoiceId (Finance domain)")
print(f"   • Transaction.InvoiceId → Invoice.InvoiceId (Finance domain)")

print(f"\n🚀 READY FOR DATA PIPELINE:")
print(f"   • Sales Order → Finance Invoice")
print(f"   • OrderPayment → Finance Payment") 
print(f"   • Payment → Finance Transaction")